### Import required libraries

In [1]:
import pickle
import gzip
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from multiprocessing import Pool
import logging

logging.basicConfig(filename="output.log",level=logging.INFO)

### Load comments corpus

In [2]:
comments_corpus = pickle.load(open("../../Results/comments_corpus.pickle","rb"))

In [3]:
comments_corpus.keys()

dict_keys(['politics', 'NeutralPolitics', 'SandersForPresident', 'The_Donald', 'hillaryclinton', 'Republican', 'democrats', 'The_Farage', 'Le_Pen', 'altright', 'progressive', 'Conservative', 'ukpolitics', 'LateStageCapitalism', 'Libertarian'])

### Define function for calculating sentiment for a specific subreddit

In [11]:
def calculate_sentiment(subreddit):
    # get only comment for the selected subreddit
    comments = comments_corpus[subreddit]
    # free up memory
    comments_corpus = None
    # init comments analyzer
    analyzer = SentimentIntensityAnalyzer()
    # inti array for storing sentiment scores
    sentiment = []
    # iterate over non-deleted and non-removed comments
    for i,comment in enumerate(comments):
        if '[deleted]' in comment or '[removed]' in comment:
            continue
        sentiment.append(analyzer.polarity_scores(comment))
        if i % 10000 == 0:
            logging.info("{} {}".format(subreddit,i))
    # serialize the result
    pickle_out = open("../../Results/Sentiment/sentiment_" + subreddit + ".pickle","wb")
    pickle.dump(sentiment, pickle_out)
    pickle_out.close()

### Run calculation and paralleise over all available cores

In [ ]:
p = Pool(len(comments_corpus))
print(p.map(calculate_sentiment, list(comments_corpus.keys())))
p.close()